In [1]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import ccxt

import src
cloudpickle.register_pickle_by_value(src) # for model portability

In [2]:

class Fetcher:
    def __init__(self, symbol=None):
        self.symbol = symbol
        self.keys = {
            'symbol': symbol
        }
        self.data_id = 'binance_ohlcv'
        
    def fetch(self, last_timestamp=None):
        client = ccxt.binance({
            'options': {
                'defaultType': 'future',
            },
        })
        
        result = client.fapiPublicGetKlines({
            'symbol': self.symbol,
            'interval': '1m',
            'startTime': 1 if last_timestamp is None else (last_timestamp + 60 * 60) * 1000,
            'limit': 1500,
        })
        
        df = pd.DataFrame(result, columns=[
            'timestamp',
            'op',
            'hi',
            'lo',
            'cl',
            'volume',
            'close_time',
            'amount',
            'trades',
            'buy_volume',
            'buy_amount',
            'ignored',
        ])
        df = df.drop(columns=['close_time', 'ignored'])
        df['timestamp'] = df['timestamp'].astype(int) // 1000
        
        for col in ['op', 'hi', 'lo', 'cl', 'volume', 'amount', 'trades', 'buy_volume', 'buy_amount']:
            df[col] = df[col].astype('float')
        
        df['timestamp_5m'] = (df['timestamp'] // 300) * 300
        df['timestamp_1h'] = (df['timestamp'] // 3600) * 3600
        
        df_5m = pd.concat([
            df.groupby('timestamp_5m')['cl'].nth(-1),
        ], axis=1)
        df_5m = df_5m.reset_index()
        df_5m['timestamp_1h'] = (df_5m['timestamp_5m'] // 3600) * 3600
        
        # display(df.loc[(df['volume'] == 0) & (df['trades'] > 0)])
        
        # microstructure feature
        # entropy feature
        # slippage feature
        
        df['hi_op'] = df['hi'] - df['op']
        df['lo_op'] = df['lo'] - df['op']
        
        df['ln_hi_lo'] = np.log(df['hi'] / df['lo'])
        df['ln_hi_lo_sqr'] = df['ln_hi_lo'] ** 2
        
#         def corwin_alpha(x):
#             hi2 = x['hi'].rolling(2).max()
#             lo2 = x['lo'].rolling(2).min()
#             gamma = np.log(hi2 / lo2) ** 2
#             beta = np.log(x['hi'] / x['lo']) ** 2
#             beta = beta.rolling(2).sum()
            
#             sqrt2 = 2.0 ** 0.5
#             denom = 3.0 - 2.0 * sqrt2
#             alpha = (sqrt2 - 1) / denom * beta ** 0.5 - (gamma / denom) ** 0.5
#             return alpha.mean()
        
        df = pd.concat([
            df.groupby('timestamp_1h')['op'].nth(0),
            df.groupby('timestamp_1h')['hi'].max(),
            df.groupby('timestamp_1h')['lo'].min(),
            df.groupby('timestamp_1h')['cl'].nth(-1),
            df.groupby('timestamp_1h')['volume'].sum(),
            df.groupby('timestamp_1h')['amount'].sum(),
            df.groupby('timestamp_1h')['trades'].sum(),
            df.groupby('timestamp_1h')['buy_volume'].sum(),
            df.groupby('timestamp_1h')['buy_amount'].sum(),
            df.groupby('timestamp_1h')['cl'].mean().rename('twap'),
            df_5m.groupby('timestamp_1h')['cl'].mean().rename('twap_5m'),
            # vola
            df.groupby('timestamp_1h')['cl'].std().fillna(0).rename('cl_std'),
            df.groupby('timestamp_1h').apply(lambda x: (x['cl'] - x['cl'].shift(1).fillna(df['op'])).std()).fillna(0).rename('cl_diff_std'),
            # slippage
            df.groupby('timestamp_1h')['hi'].mean().rename('hi_twap'),
            df.groupby('timestamp_1h')['lo'].mean().rename('lo_twap'),
            df.groupby('timestamp_1h')['hi_op'].mean().rename('hi_op_max'),
            df.groupby('timestamp_1h')['lo_op'].mean().rename('lo_op_min'),
            # microstructure
            df.groupby('timestamp_1h')['ln_hi_lo'].mean().rename('ln_hi_lo_mean'),
            df.groupby('timestamp_1h')['ln_hi_lo_sqr'].mean().rename('ln_hi_lo_sqr_mean'),
            # df.groupby('timestamp_1h').apply(corwin_alpha).fillna(0).rename('corwin_alpha'),
            # entropy
        ], axis=1)
        
        df.index.rename('timestamp', inplace=True)
        
        df = df.iloc[:-1] # remove partial
        
        return df


In [3]:
fetcher = Fetcher(symbol='BTCUSDT')
df = fetcher.fetch(last_timestamp=None)
display(df)
df = fetcher.fetch(last_timestamp=(1600000000 // 3600) * 3600)
display(df)

,op,hi,lo,cl,volume,amount,trades,buy_volume,buy_amount,twap,twap_5m,cl_std,cl_diff_std,hi_twap,lo_twap,hi_op_max,lo_op_min,ln_hi_lo_mean,ln_hi_lo_sqr_mean
timestamp,,,,,,,,,,,,,,,,,,,
1567962000,10000.00,10000.00,10000.00,10000.00,0.002,2.000000e+01,2.0,0.001,1.000000e+01,10000.000000,10000.000000,0.000000,0.000000,10000.000000,10000.000000,0.000000,0.000000,0.000000,0.000000e+00
1567965600,10000.00,10000.00,10000.00,10000.00,0.000,0.000000e+00,0.0,0.000,0.000000e+00,10000.000000,10000.000000,0.000000,0.000000,10000.000000,10000.000000,0.000000,0.000000,0.000000,0.000000e+00
1567969200,10000.00,10357.53,10000.00,10340.12,471.659,4.879792e+06,562.0,0.000,0.000000e+00,10275.480000,10287.825833,140.469630,45.084017,10276.739322,10274.418983,1.039153,-1.281186,0.000224,1.020135e-07
1567972800,10340.12,10368.64,10334.54,10351.42,583.271,6.037733e+06,731.0,0.034,3.519712e+02,10350.826833,10351.350000,8.237726,3.468910,10352.417833,10349.721167,1.388000,-1.308667,0.000261,1.084480e-07
1567976400,10351.42,10391.90,10324.77,10391.90,689.759,7.136799e+06,827.0,0.004,4.138500e+01,10347.270500,10348.556667,14.624109,3.471448,10348.277500,10345.559167,1.454167,-1.264167,0.000263,1.352056e-07
1567980000,10392.59,10412.65,10375.58,10375.58,675.394,7.020273e+06,821.0,0.000,0.000000e+00,10394.068500,10392.563333,8.151937,3.534259,10395.382833,10392.730500,1.019833,-1.632500,0.000255,1.045473e-07
1567983600,10375.58,10392.25,10366.57,10391.63,676.206,7.021684e+06,811.0,0.000,0.000000e+00,10383.645167,10385.221667,9.414928,3.261580,10384.587667,10382.231000,1.112667,-1.244000,0.000227,1.105052e-07
1567987200,10391.63,10391.63,10391.63,10391.63,0.000,0.000000e+00,0.0,0.000,0.000000e+00,10391.630000,10391.630000,0.000000,0.000000,10391.630000,10391.630000,0.000000,0.000000,0.000000,0.000000e+00
1567990800,10391.63,10391.63,10391.63,10391.63,0.000,0.000000e+00,0.0,0.000,0.000000e+00,10391.630000,10391.630000,0.000000,0.000000,10391.630000,10391.630000,0.000000,0.000000,0.000000,0.000000e+00


,op,hi,lo,cl,volume,amount,trades,buy_volume,buy_amount,twap,twap_5m,cl_std,cl_diff_std,hi_twap,lo_twap,hi_op_max,lo_op_min,ln_hi_lo_mean,ln_hi_lo_sqr_mean
timestamp,,,,,,,,,,,,,,,,,,,
1600002000,10290.28,10342.51,10260.00,10320.96,9614.284,9.907332e+07,30057.0,5159.779,5.318941e+07,10318.250167,10321.729167,12.291609,6.269115,10321.636833,10313.750167,3.874667,-4.012000,0.000765,9.908901e-07
1600005600,10320.96,10340.00,10303.93,10323.88,4805.957,4.958842e+07,19387.0,2694.436,2.780434e+07,10317.862500,10317.000000,8.703156,4.310701,10320.412833,10314.733833,2.566167,-3.112833,0.000550,3.938773e-07
1600009200,10323.89,10340.66,10273.13,10281.78,6421.050,6.616570e+07,25248.0,2885.856,2.974590e+07,10312.589833,10310.793333,14.144969,4.915285,10315.702000,10309.151667,2.422833,-4.127500,0.000636,5.844943e-07
1600012800,10281.78,10298.73,10222.00,10256.42,13596.029,1.394488e+08,37231.0,5660.461,5.806078e+07,10259.829000,10261.120833,13.310195,9.766598,10265.166000,10253.473667,4.905667,-6.786667,0.001140,1.927086e-06
1600016400,10256.43,10281.97,10209.01,10275.00,11038.868,1.130691e+08,28274.0,5195.644,5.323297e+07,10244.735667,10246.691667,16.684025,5.343475,10248.778333,10239.760833,4.344667,-4.672833,0.000880,9.523863e-07
1600020000,10274.40,10282.39,10245.01,10256.70,4110.260,4.219563e+07,17133.0,1629.694,1.673201e+07,10265.859000,10265.504167,9.600475,3.800401,10268.134667,10263.365333,1.947833,-2.821500,0.000465,2.720961e-07
1600023600,10256.69,10322.20,10247.01,10307.61,5917.699,6.089976e+07,20491.0,3552.691,3.656417e+07,10286.757333,10287.004167,20.485317,5.106946,10289.295333,10283.299333,3.365500,-2.630500,0.000583,4.849943e-07
1600027200,10307.62,10314.63,10285.60,10310.00,3742.984,3.854813e+07,16397.0,1914.328,1.971644e+07,10300.078333,10300.774167,6.568839,3.979200,10302.449667,10297.431167,2.400500,-2.618000,0.000487,3.268162e-07
1600030800,10310.00,10319.63,10287.49,10296.08,3580.750,3.690346e+07,16895.0,1822.952,1.878937e+07,10306.185333,10304.700000,7.723780,4.005225,10307.787833,10304.427500,1.380000,-1.980333,0.000326,2.186834e-07


In [4]:
symbols = 'BTC,ETH,XRP,LINK,ATOM,DOT,SOL,BNB,MATIC,ADA'.split(',')
symbols += ['ALGO', 'AVAX', 'BCH', 'APT']
fetchers = []

for symbol in symbols:
    fetchers.append(Fetcher(symbol=f'{symbol}USDT'))
    
data = cloudpickle.dumps(fetchers)
data = lzma.compress(data)
with open('/home/jovyan/data/20221103_binance_ohlcv.xz', 'wb') as f:
    f.write(data)